In [5]:
import requests     # how python goes onto the internet!
import re           # regex
from bs4 import BeautifulSoup # a python HTML parser (version 3)

In [6]:
# let's turn to stock prices
# http://finance.google.com/finance?q=aapl

In [33]:
symbol = 'amzn'

In [34]:
# Let's grab the raw html from the page
r = requests.get('http://finance.google.com/finance?q='+symbol) # the url of the google finance page goes in here

In [43]:
r
# 200 means it works!

<Response [200]>

In [36]:
# b = BeautifulSoup(r.text, "html5lib") # create a beautifulsoup object
b = BeautifulSoup(r.text, 'html.parser') # try this line instead if you have problems

In [37]:
b.prettify() # will print the html nicely

u'<!DOCTYPE html>\n<html>\n <head>\n  <script>\n   (function(){(function(){function e(a){this.t={};this.tick=function(a,c,b){var d=void 0!=b?b:(new Date).getTime();this.t[a]=[d,c];if(void 0==b)try{window.console.timeStamp("CSI/"+a)}catch(e){}};this.tick("start",null,a)}var a,d;window.performance&&(d=(a=window.performance.timing)&&a.responseStart);var f=0<d?new e(d):new e;window.jstiming={Timer:e,load:f};if(a){var c=a.navigationStart;0<c&&d>=c&&(window.jstiming.srt=d-c)}if(a){var b=window.jstiming.load;0<c&&d>=c&&(b.tick("_wtsrt",void 0,c),b.tick("wtsrt_","_wtsrt",\nd),b.tick("tbsd_","wtsrt_"))}try{a=null,window.chrome&&window.chrome.csi&&(a=Math.floor(window.chrome.csi().pageT),b&&0<c&&(b.tick("_tbnd",void 0,window.chrome.csi().startE),b.tick("tbnd_","_tbnd",c))),null==a&&window.gtbExternal&&(a=window.gtbExternal.pageT()),null==a&&window.external&&(a=window.external.pageT,b&&0<c&&(b.tick("_tbnd",void 0,window.external.startE),b.tick("tbnd_","_tbnd",c))),a&&(window.jstiming.pt=a)}catch(

In [38]:
# find all span tags
b.findAll('span')
# b.find_all('span') for bs4 users

[<span class="gbi" id="gbn"></span>,
 <span class="gbf" id="gbf"></span>,
 <span id="gbe"></span>,
 <span class="fjfe-searchbox-button-wrapper"><span class="fjfe-searchbox-button-wrapper2"><input class="fjfe-searchbox-button" tabindex="2" type="submit" value="Get quotes"/></span></span>,
 <span class="fjfe-searchbox-button-wrapper2"><input class="fjfe-searchbox-button" tabindex="2" type="submit" value="Get quotes"/></span>,
 <span class="fjfe-recentquotes-duration">(<a href="//www.google.com/history/optout?hl=en">30 days</a>)</span>,
 <span class="fjfe-chg-toggle">chg</span>,
 <span class="fjfe-perc-toggle">%</span>,
 <span class="pr">\n<span id="ref_660463_l">831.00</span>\n</span>,
 <span id="ref_660463_l">831.00</span>,
 <span class="ch bld"><span class="chr" id="ref_660463_c">-10.71</span>\n<span class="chr" id="ref_660463_cp">(-1.27%)</span>\n</span>,
 <span class="chr" id="ref_660463_c">-10.71</span>,
 <span class="chr" id="ref_660463_cp">(-1.27%)</span>,
 <span class="bld" id="r

In [39]:
# the pattern we recognized from the website
re_tag = re.compile("ref_\d+_l") 

In [40]:
#this tag finds the tag with the price in it!!!!
span_tag = b.find('span', attrs={'id': re_tag}) 
# use find to return THE ONE AND ONLY span tags with an id that matches our regex
# use findAll to find all matches

In [41]:
quote = span_tag.text
quote

u'831.00'

In [12]:
##### EXERCISE #####
# Make a method  get_stock_price that takes in ANY stock ticker and grabs the current price
# If the stock ticker doesn't exist, return -1








In [54]:
def get_stock_price(symbol):
    r = requests.get('http://finance.google.com/finance?q='+symbol)
    b = BeautifulSoup(r.text, 'html.parser')
    pattern = re.compile("ref_\d+_l") 
    span_tag = b.find('span', attrs={'id': pattern}) 
    if span_tag:
        return span_tag.text,
    else: 
        return -1


In [55]:
symbol = 'amzn'
print get_stock_price(symbol)

(u'831.00',)


In [56]:
for symbol in ['amzn','goog','goksdasdsadaogle']:
    print get_stock_price(symbol)

(u'831.00',)
(u'783.07',)
-1


In [13]:
def get_stock_price(ticker):
    response = requests.get("http://google.com/finance?q="+ticker)
    parser = BeautifulSoup(response.text, "html.parser")
    pattern = re.compile("ref_\d+_l")
    span_tag = parser.find("span", attrs={"id":pattern})
    if span_tag:
        return span_tag.text
    else:
        return -1
    
    
    

In [14]:
for ticker in ['ge', 'spy']:
    print get_stock_price(ticker)

28.83
213.15


In [15]:
###### UFO #######

In [16]:
r = requests.get("http://www.nuforc.org/webreports/ndxe201608.html")
b = BeautifulSoup(r.text, 'html.parser')

In [17]:
# Let's take a look at the first sighting
for tr in b.findAll('tr', attrs = {'valign':'TOP'})[:1]:
    # the findChildren method returns all children underneath it
    for child in tr.findChildren():
        print child.text

8/31/16 23:40
8/31/16 23:40
8/31/16 23:40
Terre Haute
Terre Haute
IN
IN
Light
Light
10 minutes
10 minutes
Four unidentified moving flashing lights that hovered in place for several minutes. Terre Haute, Indiana
Four unidentified moving flashing lights that hovered in place for several minutes. Terre Haute, Indiana
9/2/16
9/2/16


In [18]:
# OK, it's a bit messy, Let's clean it up
# Looks like the first element is the date, the 4th is the city, 6th if state, 8th is shape (this ones blank)
# 13th is the summary

In [19]:
ufo_sightings = {
        'Date':[],
        'City':[],
        'State':[],
        'Shape':[],
        'Summary':[]
    }

for tr in b.findAll('tr', attrs = {'valign':'TOP'}):
    # the findChildren method returns all children underneath it
    ufo_sighting_info = []
    for child in tr.findChildren():
        ufo_sighting_info.append(child.text)
    ufo_sightings['Date'].append(ufo_sighting_info[0])
    ufo_sightings['City'].append(ufo_sighting_info[3])
    ufo_sightings['State'].append(ufo_sighting_info[5])
    ufo_sightings['Shape'].append(ufo_sighting_info[7])
    ufo_sightings['Summary'].append(ufo_sighting_info[12])

ufo_sightings

{'City': [u'Terre Haute',
  u'Corte Madera',
  u'Malo',
  u'Arlington',
  u'Concord',
  u'Pomona',
  u'Oklahoma city',
  u'Shoreview',
  u'Memphis',
  u'Holden',
  u'Columbus',
  u'Anchorage',
  u'Spring',
  u'Menifee',
  u'Tempe',
  u'Tempe',
  u'Zeeland',
  u'Ellington',
  u'Greeley',
  u'Pittsburgh',
  u'Hyrum',
  u'Fayetteville',
  u'Dana Point',
  u'Utica',
  u'West Chicago',
  u'Ladson',
  u'Boulder',
  u'Seabrook',
  u'New York',
  u'Herriman',
  u'Newport News',
  u'Easton',
  u'Pittsfielsd',
  u'Carrollton',
  u'Arlington',
  u'Virginia Beach',
  u'Jacksonville',
  u'Surf City',
  u'Portland',
  u'St. Louis',
  u'Pomona',
  u'Columbus',
  u'Cody',
  u'Hermitage',
  u'New York City (Bronx)',
  u'New Castle',
  u'Toronto (Canada)',
  u'Newmarket (Canada)',
  u'Juneau',
  u'Runnemede',
  u'Williamsburg',
  u'Wakefield',
  u'Santa Rosa',
  u'Petaluma',
  u'Santa Rosa',
  u'Pottstown',
  u'Piscataway',
  u'Piscataway',
  u'Sylmar',
  u'Albuquerque',
  u'Northampton (UK/England)',
 

In [20]:
import pandas as pd

In [21]:
pd.DataFrame(ufo_sightings) # MAGIC

,City,Date,Shape,State,Summary
0,Terre Haute,8/31/16 23:40,Light,IN,Four unidentified moving flashing lights that ...
1,Corte Madera,8/31/16 22:00,Circle,CA,Stationary yellow gold lights seen during exte...
2,Malo,8/31/16 22:00,Light,WA,Strange meandering craft pulls near 180 before...
3,Arlington,8/31/16 21:00,Light,WI,Fifteen minute sighting of unusual light forma...
4,Concord,8/31/16 21:00,Triangle,NC,We saw 3 triangle objects in the sky with Redi...
5,Pomona,8/31/16 19:46,Sphere,CA,At 7:20pm I was in my backyard watering my gar...
6,Oklahoma city,8/31/16 16:45,Unknown,OK,Was standing outside with my boyfriend as it w...
7,Shoreview,8/31/16 12:00,Chevron,MN,While out walking I saw slow moving bright obj...
8,Memphis,8/31/16 01:38,Light,TN,I stepped outside at about 1:38 and immediatel...
9,Holden,8/30/16 23:57,Egg,MA,White/yellow glowing egg shaped object flying ...


In [22]:
# SOME MORE EXAMPLES

In [23]:
# NY TIMES ARTICLES ON HOME PAGE

In [24]:
response = requests.get("http://www.nytimes.com/")
parser = BeautifulSoup(response.text, "html.parser")
for story in parser.findAll("h2", attrs={'class':'story-heading'}):
    print story.text

Backing Away From Trump, Ryan Sets Off G.O.P. Turmoil
Risks in Renouncing Trump 
Trump Attacks Ryan and Other ‘Disloyal’ Republicans
Trump’s Threat Targets Democratic Institutions
Run-Up Podcast: On Sex and Power, From Women Covering the Campaign 10:10 AM ET
Where Republicans in Tight Races Stand on Trump 
Actually, Many ‘Inner Cities’ Are Doing Great 
Buffett Calls Trump’s Bluff and Releases Tax Data 
Early Voting Could Decide Election Before Nov. 8 
Was it a Depressing Week for Women, or a Great One? 
‘Tweet Me Your First Assaults’: A Protest Movement
Clinton Aides Kept de Blasio at Arm’s Length 

Samsung Kills Off Galaxy Note 7 Smartphone
Have a Samsung Galaxy Note 7? Here’s What to Do 1:32 PM ET
Fleeing Coast Before the Storm, Then Trapped Inland
Laying Bare the Suffering in Western Haiti
A Child in Yemen: ‘We Sleep Afraid, We Wake Up Afraid’
Ranchers Tote Guns as Tribes Dig In for Long Pipeline Fight

                                    Fighting for Water                          

In [25]:
# WIKIPEDIA FEATURED ARTICLE
response = requests.get("https://en.wikipedia.org/wiki/Wikipedia:Today%27s_featured_article")
parser = BeautifulSoup(response.text, "html.parser")

In [26]:
parser.find('div', attrs={'id':'mw-content-text'}).text

u'\n\n\nToday\'s featured article\n\nAt the top of the Main Page, a summarized lead section from one of Wikipedia\'s featured articles is displayed as "Today\'s featured article" (TFA). The current month\'s queue can be found here. TFAs are scheduled by the TFA coordinators, Brianboulton (Brian), Crisco 1492 (Chris) and Dank (Dan). Community discussion of suggestions takes place at the TFA requests page.\nIf you notice an error in a future TFA summary, you\'re welcome to fix it yourself, but if the mistake is in today\'s or tomorrow\'s summary, you can leave a message at WP:ERRORS to ask an administrator to fix it. The summaries are formatted as a single paragraph of around 1,150 characters (including spaces), with no reference tags or alternative names. Only the link to the specified featured article is bolded, and this must be the first link. The summary should be preceded by an appropriate image when available; fair use images are not allowed.\nThe editnotice template for Today\'s F